In [92]:
import pandas as pd

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, auc

In [97]:
sample_file = 'sample_submission.csv'
label_col = 'Transported'

In [104]:
def read_data():
    train_file = 'train.csv'
    test_file = 'test.csv'
    df_train = pd.read_csv(train_file)
    df_test = pd.read_csv(test_file)
    df = pd.concat([df_train, df_test], axis=0)
    return df

df = read_data()
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [145]:
df_avg_planet = df[['HomePlanet', 'Transported']].groupby('HomePlanet').mean()
df_count_planet = df[['HomePlanet', 'Transported']].groupby('HomePlanet').count()

dict(df_mean_planet['Transported'])
dict(df_count_planet['Transported'])

NameError: name 'df_mean_planet' is not defined

In [133]:
df[['HomePlanet', 'Transported']].groupby('HomePlanet').agg(['sum','count'])

Transported      
                   sum count
HomePlanet                  
Earth             1951  4602
Europa            1404  2131
Mars               920  1759

In [123]:
"""
Split PassengerID on _ into group_id, sub_id
probably passengers in the same group had similar fates
make feature: portion in group who were transported, after dropna'ing
and do the same for Cabin actually
"""
df_group_sub = df['PassengerId'].str.split('_', expand=True).rename({0: 'group_id', 1: 'sub_id'}, axis=1).astype(int)

def any_transported_in_group(df_train, group_num):
    pass

In [ ]:
def numberize_scale_dropna(df):

print(len(df_train))
df_train = df_train.dropna(subset=[label_col])
print(len(df_train))
df_train.head()

In [ ]:
df_train['Transported'].unique()

In [87]:
pd.read_csv(sample_file).head()

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False


In [88]:
y_col = 'Transported'
# x_cols = [col for col in df_train.columns if col != y_col]
x_cols = "RoomService FoodCourt ShoppingMall Spa VRDeck".split()
df_train = df_train[x_cols + [y_col]].dropna()
x_train = df_train[x_cols]
x_train = x_train.dropna()
x_train.head()

,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0.0,0.0,0.0,0.0,0.0
1,109.0,9.0,25.0,549.0,44.0
2,43.0,3576.0,0.0,6715.0,49.0
3,0.0,1283.0,371.0,3329.0,193.0
4,303.0,70.0,151.0,565.0,2.0


In [89]:
y_train = df_train[y_col].astype(bool)
y_train.head()

0    False
1     True
2    False
3    False
4     True
Name: Transported, dtype: bool

In [90]:
nbc = GaussianNB()
nbc.fit(x_train, y_train)

GaussianNB()

In [91]:
y_train_hat = nbc.predict(x_train)
confusion_matrix(y_train, y_train_hat)

array([[1551, 2322],
       [ 216, 3696]])